<a href="https://colab.research.google.com/github/nathaliemondoukpe12/app_gest_task/blob/main/traduit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Entraînement d'un modèle de traduction Anglais ↔ Français
avec MarianMT sur le dataset OPUS Books
"""

from datasets import load_dataset
from transformers import (
    MarianMTModel,
    MarianTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
import torch
import numpy as np
from tqdm import tqdm

print("=" * 70)
print("ENTRAÎNEMENT MODÈLE DE TRADUCTION EN→FR avec OPUS Books")
print("=" * 70)

# ==================== ÉTAPE 1: CHARGEMENT DU DATASET ====================

def load_opus_books(sample_size=None):
    """
    Charge le dataset OPUS Books (en-fr)

    Args:
        sample_size: nombre d'exemples à charger (None = tout)

    Returns:
        dataset
    """
    print("\n[ÉTAPE 1] Chargement du dataset OPUS Books...")

    try:
        # Chargement du dataset depuis Hugging Face
        dataset = load_dataset("opus_books", "en-fr")

        print(f"  Dataset chargé avec succès!")
        print(f"  → Train: {len(dataset['train'])} paires")

        # Échantillonnage si nécessaire (pour tester rapidement)
        if sample_size:
            dataset['train'] = dataset['train'].select(range(min(sample_size, len(dataset['train']))))
            print(f"  → Échantillon sélectionné: {len(dataset['train'])} paires")

        # Afficher quelques exemples
        print("\n   Exemples du dataset:")
        for i in range(3):
            example = dataset['train'][i]
            en = example['translation']['en']
            fr = example['translation']['fr']
            print(f"\n  Exemple {i+1}:")
            print(f"    EN: {en[:100]}...")
            print(f"    FR: {fr[:100]}...")

        return dataset

    except Exception as e:
        print(f"   Erreur lors du chargement: {e}")
        return None


# ==================== ÉTAPE 2: PRÉPARATION DES DONNÉES ====================

class DataPreprocessor:
    """Préprocesseur pour le dataset OPUS Books"""

    def __init__(self, model_name="Helsinki-NLP/opus-mt-en-fr", max_length=128):
        self.tokenizer = MarianTokenizer.from_pretrained(model_name)
        self.max_length = max_length

    def preprocess_function(self, examples):
        """Tokenise les paires de traduction"""
        # Extraire les textes anglais et français
        inputs = [ex['en'] for ex in examples['translation']]
        targets = [ex['fr'] for ex in examples['translation']]

        # Tokenisation
        model_inputs = self.tokenizer(
            inputs,
            max_length=self.max_length,
            truncation=True,
            padding="max_length"
        )

        # Tokenisation des cibles
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                targets,
                max_length=self.max_length,
                truncation=True,
                padding="max_length"
            )

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs


def prepare_dataset(dataset, preprocessor, test_size=0.1):
    """
    Prépare le dataset pour l'entraînement

    Args:
        dataset: dataset OPUS Books
        preprocessor: instance de DataPreprocessor
        test_size: proportion du set de validation

    Returns:
        train_dataset, eval_dataset
    """
    print("\n[ÉTAPE 2] Préparation des données...")

    # Séparation train/validation
    split_dataset = dataset['train'].train_test_split(test_size=test_size, seed=42)

    print(f"  → Train: {len(split_dataset['train'])} exemples")
    print(f"  → Validation: {len(split_dataset['test'])} exemples")

    # Tokenisation
    print("  → Tokenisation en cours...")
    tokenized_train = split_dataset['train'].map(
        preprocessor.preprocess_function,
        batched=True,
        remove_columns=split_dataset['train'].column_names
    )

    tokenized_eval = split_dataset['test'].map(
        preprocessor.preprocess_function,
        batched=True,
        remove_columns=split_dataset['test'].column_names
    )

    print("  ✅ Données préparées!")

    return tokenized_train, tokenized_eval


# ==================== ÉTAPE 3: CONFIGURATION DU MODÈLE ====================

def setup_model_and_trainer(tokenized_train, tokenized_eval, output_dir="./marian-en-fr"):
    """
    Configure le modèle et le trainer

    Args:
        tokenized_train: dataset d'entraînement tokenisé
        tokenized_eval: dataset de validation tokenisé
        output_dir: dossier de sortie

    Returns:
        model, trainer
    """
    print("\n[ÉTAPE 3] Configuration du modèle...")

    # Chargement du modèle pré-entraîné
    model_name = "Helsinki-NLP/opus-mt-en-fr"
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)

    print(f"  ✅ Modèle {model_name} chargé")

    # Configuration de l'entraînement
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        save_total_limit=2,
        predict_with_generate=True,
        fp16=torch.cuda.is_available(),  # Utilise FP16 si GPU disponible
        push_to_hub=False,
        logging_steps=100,
        save_steps=500,
    )

    print(f"  → Epochs: {training_args.num_train_epochs}")
    print(f"  → Batch size: {training_args.per_device_train_batch_size}")
    print(f"  → Learning rate: {training_args.learning_rate}")
    print(f"  → Device: {'GPU (FP16)' if training_args.fp16 else 'CPU'}")

    # Data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    # Création du trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    return model, trainer, tokenizer


# ==================== ÉTAPE 4: ENTRAÎNEMENT ====================

def train_model(trainer):
    """
    Entraîne le modèle

    Args:
        trainer: instance de Seq2SeqTrainer
    """
    print("\n[ÉTAPE 4] Entraînement du modèle...")
    print("  ⏳ Ceci peut prendre plusieurs heures selon votre matériel...")

    try:
        # Entraînement
        train_result = trainer.train()

        print("\n  ✅ Entraînement terminé!")
        print(f"  → Loss finale: {train_result.training_loss:.4f}")

        # Sauvegarde du modèle
        trainer.save_model()
        print("  ✅ Modèle sauvegardé!")

        return train_result

    except Exception as e:
        print(f"   Erreur pendant l'entraînement: {e}")
        return None


# ==================== ÉTAPE 5: ÉVALUATION ====================

def evaluate_model(trainer, tokenizer, test_sentences):
    """
    Évalue le modèle sur des exemples

    Args:
        trainer: instance de Seq2SeqTrainer
        tokenizer: tokenizer
        test_sentences: phrases de test
    """
    print("\n[ÉTAPE 5] Évaluation du modèle...")

    # Évaluation sur le set de validation
    eval_results = trainer.evaluate()
    print(f"\n  📊 Résultats sur le set de validation:")
    print(f"  → Loss: {eval_results['eval_loss']:.4f}")

    # Test sur des phrases personnalisées
    print("\n  🧪 Test sur des phrases personnalisées:")

    model = trainer.model
    model.eval()

    for sentence in test_sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True)

        with torch.no_grad():
            translated = model.generate(**inputs)

        translation = tokenizer.decode(translated[0], skip_special_tokens=True)

        print(f"\n  EN: {sentence}")
        print(f"  FR: {translation}")


# ==================== FONCTION PRINCIPALE ====================

def main(sample_size=5000):
    """
    Pipeline complet d'entraînement

    Args:
        sample_size: nombre d'exemples à utiliser (None = tous)
    """

    # 1. Charger le dataset
    dataset = load_opus_books(sample_size=sample_size)
    if dataset is None:
        return

    # 2. Préparer les données
    preprocessor = DataPreprocessor(max_length=128)
    tokenized_train, tokenized_eval = prepare_dataset(dataset, preprocessor)

    # 3. Configurer le modèle
    model, trainer, tokenizer = setup_model_and_trainer(
        tokenized_train,
        tokenized_eval,
        output_dir="./marian-en-fr-finetuned"
    )

    # 4. Entraîner
    train_result = train_model(trainer)

    if train_result is None:
        return

    # 5. Évaluer
    test_sentences = [
        "Hello, how are you?",
        "I love reading books.",
        "The weather is beautiful today.",
        "Machine learning is fascinating.",
        "Thank you very much for your help."
    ]

    evaluate_model(trainer, tokenizer, test_sentences)

    print("\n" + "=" * 70)
    print("✅ ENTRAÎNEMENT TERMINÉ!")
    print("=" * 70)
    print(f"\nModèle sauvegardé dans: ./marian-en-fr-finetuned")
    print("\nPour utiliser le modèle:")
    print("  model = MarianMTModel.from_pretrained('./marian-en-fr-finetuned')")
    print("  tokenizer = MarianTokenizer.from_pretrained('./marian-en-fr-finetuned')")


# ==================== GUIDE D'UTILISATION ====================

print("""
📦 PRÉREQUIS - INSTALLATION:

pip install transformers datasets torch sentencepiece sacremoses

💡 UTILISATION:

1. ENTRAÎNEMENT RAPIDE (échantillon de 5000 exemples):
   main(sample_size=5000)

2. ENTRAÎNEMENT COMPLET (tout le dataset):
   main(sample_size=None)

⚙️ PARAMÈTRES À AJUSTER:

- sample_size: nombre d'exemples (5000 pour test rapide)
- num_train_epochs: nombre d'époques (3 par défaut)
- per_device_train_batch_size: taille du batch (8 par défaut)
- learning_rate: taux d'apprentissage (5e-5 par défaut)
- max_length: longueur max des séquences (128 par défaut)

⏱️ TEMPS D'ENTRAÎNEMENT ESTIMÉ:

- CPU: ~6-8 heures pour 5000 exemples
- GPU (NVIDIA): ~30-60 min pour 5000 exemples
- Dataset complet: plusieurs heures même avec GPU

💾 ESPACE DISQUE NÉCESSAIRE:

- Dataset: ~500 MB
- Modèle: ~300 MB
- Total: ~1 GB

  POUR DÉBUTER:
   Commencez avec sample_size=1000 pour tester rapidement!
""")

print("\n" + "=" * 70)
print("🚀 LANCER L'ENTRAÎNEMENT")
print("=" * 70)
print("\nPour démarrer avec un échantillon de 5000 exemples:")
print(">>> main(sample_size=5000)")
print("\nPour utiliser tout le dataset:")
print(">>> main(sample_size=None)")

# Décommenter pour lancer l'entraînement automatiquement:
# if __name__ == "__main__":
#     main(sample_size=5000)

ENTRAÎNEMENT MODÈLE DE TRADUCTION EN→FR avec OPUS Books

📦 PRÉREQUIS - INSTALLATION:

pip install transformers datasets torch sentencepiece sacremoses

💡 UTILISATION:

1. ENTRAÎNEMENT RAPIDE (échantillon de 5000 exemples):
   main(sample_size=5000)

2. ENTRAÎNEMENT COMPLET (tout le dataset):
   main(sample_size=None)

⚙️ PARAMÈTRES À AJUSTER:

- sample_size: nombre d'exemples (5000 pour test rapide)
- num_train_epochs: nombre d'époques (3 par défaut)
- per_device_train_batch_size: taille du batch (8 par défaut)
- learning_rate: taux d'apprentissage (5e-5 par défaut)
- max_length: longueur max des séquences (128 par défaut)

⏱️ TEMPS D'ENTRAÎNEMENT ESTIMÉ:

- CPU: ~6-8 heures pour 5000 exemples
- GPU (NVIDIA): ~30-60 min pour 5000 exemples
- Dataset complet: plusieurs heures même avec GPU

💾 ESPACE DISQUE NÉCESSAIRE:

- Dataset: ~500 MB
- Modèle: ~300 MB
- Total: ~1 GB

🎯 POUR DÉBUTER:
   Commencez avec sample_size=1000 pour tester rapidement!


🚀 LANCER L'ENTRAÎNEMENT

Pour démarrer av